In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import count, col
from pyspark.sql.functions import struct
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()
import pandas as pd
#df1 = df.groupBy("name", "genre").count()
#f2 = df1.groupby('name').agg(F.max("count").alias("max_count"))
#Now trying to join both dataframes

IllegalArgumentException: "Error while instantiating 'org.apache.spark.sql.hive.HiveSessionStateBuilder':"

In [132]:
Benefit=spark.read.csv(r"D:\Others\DB\new\benefits_clean.csv", header=True,inferSchema=True)
Rate=spark.read.csv(r"D:\Others\DB\new\Rate_Cleaned.csv", header=True,inferSchema=True)

In [133]:
Rate=Rate.replace("0-20","10","Age")
Rate=Rate.withColumn("Age", Rate["Age"].cast(IntegerType()))

In [134]:
#df = Rate.join(Benefit,[Rate.PlanId == Benefit.StandardComponentId])
df=Rate.join(Benefit, Benefit.StandardComponentId == Rate.PlanId, 'inner').select(Rate.PlanId,Rate.IndividualRate, Rate.Age ,Benefit.BusinessYear,Benefit.BenefitName,Benefit.CoinsInnTier1Value,Benefit.CoinsInnTier2Value,Benefit.CoinsOutofNetValue,Benefit.LimitQty)

In [135]:
df=df.filter(df["Age"]!=99)
#df=df.na.replace(21,19)
#from pyspark.sql.types import IntegerType
#df=df.withColumn("Age", df["Age"].cast(IntegerType()))

In [136]:
#df.count()
df2014=df.filter(df["BusinessYear"]==2014)
df2015=df.filter(df["BusinessYear"]==2015)
df2016=df.filter(df["BusinessYear"]==2016)

In [137]:
from pyspark.sql import functions as func
tmp=df2015.filter(df["CoinsInnTier1Value"]!=-1)
t1_15=tmp.groupby('PlanId').agg({'CoinsInnTier1Value':'mean'})
tmp=df2015.filter(df["CoinsInnTier2Value"]!=-1)
t2_15=tmp.groupby('PlanId').agg({'CoinsInnTier2Value':'mean'})
tmp=df2015.filter(df["CoinsOutofNetValue"]!=-1)
t3_15=tmp.groupby('PlanId').agg({'CoinsOutofNetValue':'mean'})

tmp=df2015
t4_15=tmp.groupby('PlanId').agg({'LimitQty':'mean'})

tmp=df2015.filter(df["Age"]!=-1)
t5_15=tmp.groupby('PlanId').agg({'Age':'mean'})


tmp=df2015.filter(df["IndividualRate"]!=-1)
t6_15=tmp.groupby('PlanId').agg({'IndividualRate':'mean'})

In [138]:
data_2015=t1_15.join(t2_15, t2_15.PlanId == t1_15.PlanId, 'inner').drop(t1_15.PlanId)
data_2015=data_2015.join(t3_15, t3_15.PlanId == data_2015.PlanId, 'inner').drop(data_2015.PlanId)
data_2015=data_2015.join(t4_15, t4_15.PlanId == data_2015.PlanId, 'inner').drop(data_2015.PlanId)
data_2015=data_2015.join(t5_15, t5_15.PlanId == data_2015.PlanId, 'inner').drop(data_2015.PlanId)
data_2015=data_2015.join(t6_15, t6_15.PlanId == data_2015.PlanId, 'inner').drop(data_2015.PlanId)

In [139]:
data_2015.show()

+-----------------------+-----------------------+-----------------------+------------------+-----------------+--------------+-------------------+
|avg(CoinsInnTier1Value)|avg(CoinsInnTier2Value)|avg(CoinsOutofNetValue)|     avg(LimitQty)|         avg(Age)|        PlanId|avg(IndividualRate)|
+-----------------------+-----------------------+-----------------------+------------------+-----------------+--------------+-------------------+
|    0.19655172413786762|    0.21133004926101276|     0.4014778325123153| 3106.785714285714|41.77777777777778|29276OH0740029| 439.40064705836517|
|    0.20000000000000856|    0.29999999999991633|    0.29999999999991633|               1.0|41.77777777777778|34930MI0110003| 32.721874999995705|
|    0.16438356164382462|    0.16438356164382462|     0.5342465753424658|            26.625|41.77777777777778|34996NV0070003| 300.83433478223344|
|    0.30000000000001287|    0.34999999999994524|    0.34999999999994524|               1.0|41.77777777777778|56891NC0060003